# Header Section

## Assignment Title: Specialized Wikipedia Research Assistant

## Research Domain: Techonology - Artificial Intelligence
The selected technology domain is subfield of Artifical Intelligence. This field is chosen because of many of these topics are covered during my undergradaute and my current graduate studies, however during my udergraduate many of these topics are thought by themselves. For example, state search algorthims, while interesting and helped me solve games like tower of hanoi, backgammon, are not that connected to things like traditional machine learning, deep learning etc. I found it difficult to link these topics during my undergraduate and after awhile, forgot about these concept. This research assistance may be able to help breach this gap.

## Link of 5 Wikipedia websites
https://en.wikipedia.org/wiki/Agentic_AI <br>
https://en.wikipedia.org/wiki/Reinforcement_learning <br>
https://en.wikipedia.org/wiki/Machine_learning <br>
https://en.wikipedia.org/wiki/Deep_learning <br>
https://en.wikipedia.org/wiki/Natural_language_processing <br>





# Setup and Configuration
Install neccessary packages and set up API (OpenAI)

In [ ]:
!pip install crawl4ai>=0.2.0 openai>=1.0.0 pydantic>=2.0.0 python-dotenv>=1.0.0 requests>=2.25.0 beautifulsoup4>=4.9.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.1.0 which is incompatible.


In [ ]:
!playwright install

173.7 MiB [] 0% 132.2s173.7 MiB [] 0% 19.3s173.7 MiB [] 0% 6.2s173.7 MiB [] 1% 4.8s173.7 MiB [] 1% 4.4s173.7 MiB [] 1% 4.3s173.7 MiB [] 2% 4.0s173.7 MiB [] 2% 3.8s173.7 MiB [] 3% 3.6s173.7 MiB [] 3% 3.7s173.7 MiB [] 4% 3.6s173.7 MiB [] 4% 3.5s173.7 MiB [] 5% 3.5s173.7 MiB [] 5% 4.1s173.7 MiB [] 6% 3.9s173.7 MiB [] 7% 3.7s173.7 MiB [] 8% 3.4s173.7 MiB [] 9% 3.3s173.7 MiB [] 9% 3.2s173.7 MiB [] 10% 3.0s173.7 MiB [] 11% 2.9s173.7 MiB [] 12% 2.7s173.7 MiB [] 13% 2.6s173.7 MiB [] 14% 2.5s173.7 MiB [] 15% 2.4s173.7 MiB [] 16% 2.3s173.7 MiB [] 17% 2.3s173.7 MiB [] 18% 2.2s173.7 MiB [] 19% 2.1s173.7 MiB [] 20% 2.0s173.7 MiB [] 21% 1.9s173.7 MiB [] 22% 1.9s173.7 MiB [] 23% 1.8s173.7 MiB [] 24% 1.8s173.7 MiB [] 25% 1.8s173.7 MiB [] 26% 1.8s173.7 MiB [] 27% 1.8s173.7 MiB [] 28% 1.8s173.7 MiB [] 29% 1.8s173.7 MiB [] 30% 1.7s173.7 MiB [] 31% 1.7s173.7 MiB [] 32% 1.7s173.7 MiB [] 33% 1.7s173.7 MiB [] 34% 1.6s173.7 MiB [] 35% 1.6s173.7 MiB [] 36% 1.6s173.7 MiB [] 37% 1.6s173.7 MiB [] 38% 1.6s173.7 Mi

# Import Libraries and set up openAI

In [ ]:
import asyncio
import json
import time
from typing import List, Dict, Any
import pandas as pd
import numpy as np
import re
import os
import getpass
from concurrent.futures import ThreadPoolExecutor



# Crawl4AI imports
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy, LLMExtractionStrategy
from crawl4ai.chunking_strategy import RegexChunking, SlidingWindowChunking
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
from crawl4ai import CacheMode

# LLM and embedding imports
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

## API handling

In [ ]:
# Configuration - Update with your API keys
api_key = getpass.getpass(prompt="Enter your OpenAI API key (it will not be shown as you type): ")
# Set the environment variable with the key
os.environ['OPENAI_API_KEY'] = api_key

def create_secure_openai_client():
    """
    Create OpenAI client with secure API key handling.

    This function:
    1. Looks for OPENAI_API_KEY in environment variables
    2. Tests the connection with a simple API call
    3. Returns the client or None if setup fails
    """
    try:
        from dotenv import load_dotenv
        load_dotenv()  # Load .env file if it exists
    except ImportError:
        pass  # python-dotenv not installed, that's okay

    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        print("⚠️ No OpenAI API key found.")
        print("💡 Set environment variable: OPENAI_API_KEY=your_key")
        print("💡 Or create .env file with: OPENAI_API_KEY=your_key")
        return None

    try:
        client = OpenAI(api_key=api_key)
        # Test connection with a simple API call
        models = client.models.list()
        print("✅ OpenAI client created and tested successfully")
        return client
    except Exception as e:
        print(f"❌ OpenAI client creation failed: {e}")
        print("🔍 Check your API key and internet connection")
        return None

# Initialize the client
client = create_secure_openai_client()

Enter your OpenAI API key (it will not be shown as you type): ··········
✅ OpenAI client created and tested successfully


# Part 1 Wikipedia Scraper Implementation
The following below is the WikipediaScraper class, it has a list of urls to scrape. Some cleaning based on Wikipedia's raw markdown format is done in the clean_content method. In this case, the 5 URL are mentioned above
## Link of 5 Wikipedia websites
https://en.wikipedia.org/wiki/Agentic_AI <br>
https://en.wikipedia.org/wiki/Reinforcement_learning <br>
https://en.wikipedia.org/wiki/Machine_learning <br>
https://en.wikipedia.org/wiki/Deep_learning <br>
https://en.wikipedia.org/wiki/Natural_language_processing <br>

## Explanation of how articles relate to research domain
All aritcles are subclasses of Artifical Intelligence.




In [ ]:
class WikipediaScraper:
  def __init__(self, base_urls: List[str]):
    self.base_urls = base_urls

  async def scrape_article(self, url: str) -> Dict[str, Any]:
    """Scrape a single Wikipedia article"""

    # Configure optimized crawling
    chunking_strategy = SlidingWindowChunking(
        window_size=1000,    # Optimal for LLM processing
        step=500            # 50% overlap for context preservation
    )

    config = CrawlerRunConfig(
        markdown_generator=DefaultMarkdownGenerator(),
        chunking_strategy=chunking_strategy,
        cache_mode=CacheMode.WRITE_ONLY,  # results of this crawl are saved to the cache
        page_timeout=15000,  # Faster timeout for efficiency
        verbose=False       # Reduce logging for speed
    )
    async with AsyncWebCrawler() as crawler:
      try:
        result = await crawler.arun(url=url, config=config)
        if result.success:
          raw_content = result.markdown.raw_markdown
          cleaned_content = self.clean_content(raw_content)
          return {
              "url": url,
              "title": result.metadata.get("title", "Unknown"),
              "raw_content": cleaned_content,
              "success": True
          }
        else:
          print(f"   ❌ Failed: {result.error_message}")
          return {
              "url": url,
              "title": "Unknown",
              "raw_content": result.error_message,
              "success": False,
          }
      except Exception as e:
        print(f"   ❌ Exception: {e}")
        return {
            "url": url,
            "title": "Unknown",
            "raw_content": str(e),
            "success": False,
        }


  async def scrape_multiple(self) -> List[Dict[str, Any]]:
    """Scrape all articles in the URL list"""
    results =[]
    start_time= time.time()
    for url in self.base_urls:
      print(f"⚡ Scraping: {url}")
      article_result = await self.scrape_article(url)
      results.append(article_result)
    elapsed = time.time() - start_time
    print(f"✅ Finished scraping {len(self.base_urls)} articles in {elapsed:.2f} seconds")

    return results

  def clean_content(self, raw_content: str) -> str:
    """Clean and preprocess scraped content"""
    text = raw_content

    # 1. Keep only from the first H1 (# Title) onwards
    match = re.search(r"^# .+", text, flags=re.MULTILINE)
    if match:
        text = text[match.start():]

    # 2. Remove navigation / sidebar / account / tools sections by keyword
    remove_keywords = [
        "Main menu", "Navigation", "Contribute", "Appearance", "Personal tools",
        "Pages for logged out editors", "Toggle the table of contents",
        "Tools", "Print/export", "In other projects",
        "Edit links", "Create account", "Log in", "Donate",
        "move to sidebar hide", "Search", "Width", "Color (beta)", "Actions",
    ]
    pattern = re.compile(rf"^.*({'|'.join(map(re.escape, remove_keywords))}).*$",
                         re.MULTILINE | re.IGNORECASE)
    text = pattern.sub("", text)

    # 3. Remove [edit] links
    text = re.sub(r"\[\[edit.*?\]\]", "", text, flags=re.IGNORECASE)


    # 4. Remove inline citation tooltips (#cite_note stuff)
    text = re.sub(r"\[\d+\]\(https:\/\/en\.wikipedia\.org\/wiki\/[^)]+#cite_note[^\)]*\)",
                  lambda m: "[" + re.search(r"\d+", m.group()).group() + "]",
                  text)

    # 5. Remove excess blank lines
    text = re.sub(r"\n{2,}", "\n\n", text)

    return text.strip()
    # chunks = raw_content.split('\n\n')

    # # Quality filter: only keep substantial chunksq
    # quality_chunks = [
    #     chunk for chunk in chunks
    #     if len(chunk.strip()) > 100 and
    #     not chunk.strip().startswith(('cookie', 'javascript', 'advertisement', 'navigation', 'contribute'))
    # ]
    # # Filter out junk lines
    # def regex_clean(text:str) -> str:
    #   text = re.sub(r"\[edit\]", "", text)
    #   text = re.sub(r"\[\d+\]", "", text)
    #   text = re.sub(r"\{\{.*?\}\}", "", text, flags=re.DOTALL)
    #   text = re.sub(r"Category:.*", "", text)
    #   text = re.sub(r"\n{2,}", "\n", text)
    #   return text.strip()

    # cleaned_quality_chunks = [regex_clean(c) for c in quality_chunks]

    # return quality_chunks

## Demonstration of Wikipedia Scraper
Below is a simple demonstation, the 5 links are used to create this scraper class, and results are saved into scrapper_result, which is a list of dictionary and each dictionary has 4 fields


```
{
  "url": url,
  "title": result.metadata.get("title", "Unknown"),
  "raw_content": cleaned_content,
  "success": True
}
```



In [ ]:
ai_scraper = WikipediaScraper(["https://en.wikipedia.org/wiki/Agentic_AI", "https://en.wikipedia.org/wiki/Reinforcement_learning", "https://en.wikipedia.org/wiki/Machine_learning", "https://en.wikipedia.org/wiki/Deep_learning", "https://en.wikipedia.org/wiki/Natural_language_processing"])
scrapper_result = await ai_scraper.scrape_multiple()

⚡ Scraping: https://en.wikipedia.org/wiki/Agentic_AI


[INIT].... → Crawl4AI 0.7.4 

[COMPLETE] ● Database backup created at: /root/.crawl4ai/crawl4ai.db.backup_20250901_061440 

[INIT].... → Starting database migration... 

[COMPLETE] ● Migration completed. 0 records processed. 

...

⚡ Scraping: https://en.wikipedia.org/wiki/Deep_learning


[INIT].... → Crawl4AI 0.7.4 

⚡ Scraping: https://en.wikipedia.org/wiki/Natural_language_processing


[INIT].... → Crawl4AI 0.7.4 

✅ Finished scraping 5 articles in 21.62 seconds


In [ ]:
for key, value in scrapper_result[0].items():
  if key != "raw_content":
    print(f"{key} : {value}")
  else:
    print(f"{key} : {value[:2000] + "..."}")

url : https://en.wikipedia.org/wiki/Agentic_AI
title : Agentic AI - Wikipedia
raw_content : # Agentic AI
4 languages
  * [Català](https://ca.wikipedia.org/wiki/IA_ag%C3%A8ntica "IA agèntica – Catalan")
  * [فارسی](https://fa.wikipedia.org/wiki/%D9%87%D9%88%D8%B4_%D9%85%D8%B5%D9%86%D9%88%D8%B9%DB%8C_%D8%AE%D9%88%D8%AF%D9%85%D8%AE%D8%AA%D8%A7%D8%B1 "هوش مصنوعی خودمختار – Persian")
  * [한국어](https://ko.wikipedia.org/wiki/%EC%97%90%EC%9D%B4%EC%A0%84%ED%8B%B1_AI "에이전틱 AI – Korean")
  * [Русский](https://ru.wikipedia.org/wiki/%D0%90%D0%B3%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D0%98%D0%98 "Агентный ИИ – Russian")

  * [Article](https://en.wikipedia.org/wiki/Agentic_AI "View the content page \[alt-c\]")
  * [Talk](https://en.wikipedia.org/wiki/Talk:Agentic_AI "Discuss improvements to the content page \[alt-t\]")

English
  * [Read](https://en.wikipedia.org/wiki/Agentic_AI)
  * [Edit](https://en.wikipedia.org/w/index.php?title=Agentic_AI&action=edit "Edit this page \[alt-e\]")
  * [View history]

### Example of error handling
Below is an example of error handling, in the list of URL, even though 1 of the website is a fake website, the correct one still gets scrapped

In [ ]:
#Negative example and error handling for wrong URLs
bad_scrapper = WikipediaScraper(["notawebsite.com","https://en.wikipedia.org/wiki/Agentic_AI"])
bad_scrapper_result = await bad_scrapper.scrape_multiple()

⚡ Scraping: notawebsite.com


[INIT].... → Crawl4AI 0.7.4 

   ❌ Failed: Unexpected error in _crawl_web at line 500 in crawl (../usr/local/lib/python3.12/dist-packages/crawl4ai/async_crawler_strategy.py):
Error: URL must start with 'http://', 'https://', 'file://', or 'raw:'

Code context:
 495                   status_code=status_code,
 496                   screenshot=screenshot_data,
 497                   get_delayed_content=None,
 498               )
 499           else:
 500 →             raise ValueError(
 501                   "URL must start with 'http://', 'https://', 'file://', or 'raw:'"
 502               )
 503   
 504       async def _crawl_web(
 505           self, url: str, config: CrawlerRunConfig
⚡ Scraping: https://en.wikipedia.org/wiki/Agentic_AI


[INIT].... → Crawl4AI 0.7.4 

✅ Finished scraping 2 articles in 4.46 seconds


In [ ]:
bad_scrapper_result[0]

{'url': 'notawebsite.com',
 'title': 'Unknown',
 'raw_content': 'Unexpected error in _crawl_web at line 500 in crawl (../usr/local/lib/python3.12/dist-packages/crawl4ai/async_crawler_strategy.py):\nError: URL must start with \'http://\', \'https://\', \'file://\', or \'raw:\'\n\nCode context:\n 495                   status_code=status_code,\n 496                   screenshot=screenshot_data,\n 497                   get_delayed_content=None,\n 498               )\n 499           else:\n 500 →             raise ValueError(\n 501                   "URL must start with \'http://\', \'https://\', \'file://\', or \'raw:\'"\n 502               )\n 503   \n 504       async def _crawl_web(\n 505           self, url: str, config: CrawlerRunConfig',
 'success': False}

# Part 2 Structured Data Extraction
The following is the structured data extraction portion.
## pydantic class
First we create a pydantic class. To answer the required questions, 7 fields are chosen.


```
main_topic, summary, evolution_timeline, key_innovations, major_contributors, main_techniques, applications
```


In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional

class WikipediaExtraction(BaseModel):
  """Schema for artificial intelligence subfield"""
  main_topic: str = Field(description="The name of the AI subfield (e.g., Machine Learning, Expert Systems, Computer Vision)")
  summary: str = Field(desciption="Concise overview of the subfield, its purpose, and scope")
  evolution_timeline: str = Field(description="Chronological evolution of this subfield, with notable milestones across time")
  key_innovations: str = Field(description="The most important breakthroughs, inventions, or discoveries that shaped this subfield")
  major_contributors: str = Field(description="Key researchers, organizations, or companies that significantly advanced this subfield")
  main_techniques: List[str] = Field(description="Core methods, models, or algorithms commonly used in this subfield")
  applications: List[str] = Field(description="Practical use cases and domains where this subfield has had major impact")

# Let's see what the JSON schema looks like
print("📋 Generated JSON Schema Preview:")
print(json.dumps(WikipediaExtraction.model_json_schema(), indent=2))

📋 Generated JSON Schema Preview:
{
  "description": "Schema for artificial intelligence subfield",
  "properties": {
    "main_topic": {
      "description": "The name of the AI subfield (e.g., Machine Learning, Expert Systems, Computer Vision)",
      "title": "Main Topic",
      "type": "string"
    },
    "summary": {
      "desciption": "Concise overview of the subfield, its purpose, and scope",
      "title": "Summary",
      "type": "string"
    },
    "evolution_timeline": {
      "description": "Chronological evolution of this subfield, with notable milestones across time",
      "title": "Evolution Timeline",
      "type": "string"
    },
    "key_innovations": {
      "description": "The most important breakthroughs, inventions, or discoveries that shaped this subfield",
      "title": "Key Innovations",
      "type": "string"
    },
    "major_contributors": {
      "description": "Key researchers, organizations, or companies that significantly advanced this subfield",
     

## Extraction class
The following is the extraction class, cacheing and rate limiter are used for an optimized extraction.

In [ ]:
class OptimizedProcessor:
    """
    Optimized processor for high-throughput applications.

    Features:
    - Batch processing for efficiency
    - Async operations for concurrent requests
    - Caching to reduce API calls
    - Rate limiting to respect API limits
    """

    def __init__(self,client:Optional[OpenAI] = None):
      self.client = client
      self.cache = {}
      self.rate_limiter = RateLimiter(requests_per_minute=60)



    def extract_strucutred_data(self, content: str, model: str = "gpt-4o-mini") -> WikipediaExtraction:
    #create schema for Open AI
      schema = {
          "name": "AI_subfield_catalog",
          "schema": {
              **WikipediaExtraction.model_json_schema(),
              "additionalProperties": False
          },
          "strict": True  # This enforces strict schema compliance
      }
      try:
        # The magic happens here - response_format enforces our schema
        response = self.client.chat.completions.create(
            model=model,  # Only gpt-4o and gpt-4o-mini support structured outputs
            messages=[
                {
                    "role": "system",
                    "content": "Extract AI subfield information from text and structure it exactly according to the provided schema."
                },
                {
                    "role": "user",
                    "content": f"Extract and analyze AI subfield data from:\n\n{content}"
                }
            ],
            response_format={
                "type": "json_schema",
                "json_schema": schema
            }
        )
        # Direct validation - no parsing errors possible!
        ai_subfield = WikipediaExtraction.model_validate_json(response.choices[0].message.content)

        print("✅ Structured extraction successful!")
        return ai_subfield
      except Exception as e:
            print(f"❌ Extraction failed: {e}")
            return None

    def batch_extract(self, articles: List[Dict]) -> List[WikipediaExtraction]:
      final_dic= {}
      for dic in articles:
        current_title = dic["title"]
        # Check cache first
        cache_key = hash(dic["raw_content"][:100])
        if cache_key in self.cache:
          print("💾 Using cached result")
          final_dic[self.cache[cache_key].main_topic] = (self.cache[cache_key])
          continue
        # Rate limiting
        self.rate_limiter.wait_if_needed()
        #Process content
        result = self.extract_strucutred_data(dic["raw_content"])
        if result:
          self.cache[cache_key] = result
          final_dic[result.main_topic] = result
        else:
          print(f"Extraction failed for {current_title}")
      return final_dic

class RateLimiter:
    """Simple rate limiter for API calls."""

    def __init__(self, requests_per_minute: int):
        self.requests_per_minute = requests_per_minute
        self.last_request_time = 0

    def wait_if_needed(self):
        """Wait if we're hitting rate limits."""
        current_time = time.time()
        min_interval = 60.0 / self.requests_per_minute

        if current_time - self.last_request_time < min_interval:
            wait_time = min_interval - (current_time - self.last_request_time)
            time.sleep(wait_time)

        self.last_request_time = time.time()


In [ ]:
# Demonstrate optimized processing
optimized_processor = OptimizedProcessor(client)
print("⚡ Optimized processor created with batching and caching")
structured_outputs = optimized_processor.batch_extract(scrapper_result)

⚡ Optimized processor created with batching and caching
✅ Structured extraction successful!
✅ Structured extraction successful!
✅ Structured extraction successful!
✅ Structured extraction successful!
✅ Structured extraction successful!


In [ ]:
# Demonstrate error handling, if no client is parsed
optimized_processor_missing_client = OptimizedProcessor()
print("⚡ Optimized processor created with batching and caching")
structured_outputs_missing_client = optimized_processor_missing_client.batch_extract(scrapper_result)

⚡ Optimized processor created with batching and caching
❌ Extraction failed: 'NoneType' object has no attribute 'chat'
Extraction failed for Agentic AI - Wikipedia
❌ Extraction failed: 'NoneType' object has no attribute 'chat'
Extraction failed for Reinforcement learning - Wikipedia
❌ Extraction failed: 'NoneType' object has no attribute 'chat'
Extraction failed for Machine learning - Wikipedia
❌ Extraction failed: 'NoneType' object has no attribute 'chat'
Extraction failed for Deep learning - Wikipedia
❌ Extraction failed: 'NoneType' object has no attribute 'chat'
Extraction failed for Natural language processing - Wikipedia


## Analyze structued outputs:
The following code shows the output after extraction and what patterns to expect. Remember we have 7 field
```
main_topic, summary, evolution_timeline, key_innovations, major_contributors, main_techniques, applications
```
The structured outputs is a dictionary where `main_topic` is used as the key. This seems to be better than crawl4ai's title, at least in Wikipedia case. Instead of giving `"Agentic AI - Wikipedia"`, the extracted `main_topic` gives `"Agentic AI"`

In [ ]:
[item for item in structured_outputs.keys()]

['Agentic AI',
 'Reinforcement Learning',
 'Machine Learning',
 'Deep Learning',
 'Natural Language Processing']

In [ ]:
print(f'summary: {structured_outputs["Machine Learning"].summary}')
print(f'evolution_timeline: {structured_outputs["Machine Learning"].evolution_timeline}')
print(f'key_innovations: {structured_outputs["Machine Learning"].key_innovations}')
print(f'major_contributors: {structured_outputs["Machine Learning"].major_contributors}')
print(f'main_techniques: {structured_outputs["Machine Learning"].main_techniques}')
print(f'applications: {structured_outputs["Machine Learning"].applications}')

summary: Machine learning (ML) is a subfield of artificial intelligence that focuses on creating systems that can learn from and make predictions based on data. It encompasses a variety of techniques and theories that allow computers to improve their performance on tasks through experience.
evolution_timeline: • 1959: The term 'machine learning' is coined by Arthur Samuel. 
• 1990s: Focus shifts from achieving artificial intelligence to practical solvable problems. 
• 2010s: Deep learning emerges, leveraging neural networks for higher accuracy in various applications.
key_innovations: • Development of decision trees. 
• Introduction of neural networks and backpropagation. 
• Emergence of support vector machines. 
• Development of deep learning architectures.
major_contributors: • Arthur Samuel, Tom Mitchell, Geoffrey Hinton, Yann LeCun, Andrew Ng, Fei-Fei Li.
main_techniques: ['Supervised learning', 'Unsupervised learning', 'Reinforcement learning', 'Decision trees', 'Support vector ma

# Part 3 Function calling implementaion
The following codes are function calling implementation, `compare_technologies` and `trace_evolution`
`tech1`, `tech2` and `technology` are the arguments, later in the schema we bound them to the keys in our structured_outputs

In [ ]:
def compare_technologies(tech1:str, tech2:str):
  """Compare two technologies"""
  tech1_output = structured_outputs[tech1]
  tech2_output = structured_outputs[tech2]
  if not tech1_output or not tech2_output:
    return "One or both technologies not found."
  comparison_prompt = f"""
  Compare the following technologies based on their summary, main_techniques ,applications

  {tech1_output.main_topic}:
  Summary: {tech1_output.summary}
  Main Techniques: {tech1_output.main_techniques}
  Applications: {tech1_output.applications}

  {tech2_output.main_topic}:
  Summary: {tech2_output.summary}
  Main Techniques: {tech2_output.main_techniques}
  Applications: {tech2_output.applications}
  """
  #search first for the topic
  return comparison_prompt

def trace_evolution(technology:str):
  """Trace the evolution of a technology"""
  tech_output = structured_outputs[technology]
  if not tech_output:
    return "Technology not found."
  evolution_prompt = f"""
  Trace the evolution of the following technology:

  {tech_output.main_topic}:
  Summary: {tech_output.summary}
  Evolution Timeline: {tech_output.evolution_timeline}
  Major Contributors: {tech_output.major_contributors}
  Key Innovations: {tech_output.key_innovations}
  """
  return evolution_prompt

In [ ]:
print(compare_technologies("Agentic AI", "Machine Learning"))


  Compare the following technologies based on their summary, main_techniques ,applications

  Agentic AI:
  Summary: Agentic AI is a class of artificial intelligence focused on autonomous systems that can make decisions and perform tasks with or without human intervention, utilizing various AI techniques such as natural language processing and machine learning.
  Main Techniques: ['Natural Language Processing', 'Machine Learning', 'Computer Vision', 'Robotic Process Automation']
  Applications: ['Software Development', 'Customer Support', 'Cybersecurity', 'Business Intelligence', 'Web Browsing']

  Machine Learning:
  Summary: Machine learning (ML) is a subfield of artificial intelligence that focuses on creating systems that can learn from and make predictions based on data. It encompasses a variety of techniques and theories that allow computers to improve their performance on tasks through experience.
  Main Techniques: ['Supervised learning', 'Unsupervised learning', 'Reinforcemen

In [ ]:
print(compare_technologies("Deep Learning", "Machine Learning"))


  Compare the following technologies based on their summary, main_techniques ,applications

  Deep Learning:
  Summary: Deep Learning is a subfield of machine learning that focuses on utilizing multilayered neural networks to perform tasks such as classification, regression, and representation learning. It imitates how human brains operate by stacking artificial neurons and training them to process massive amounts of data, often leading to breakthroughs in various domains, particularly in computer vision, speech recognition, and natural language processing.
  Main Techniques: ['Neural Networks', 'Convolutional Neural Networks (CNNs)', 'Recurrent Neural Networks (RNNs)', 'Transformers', 'Generative Adversarial Networks (GANs)', 'Deep Belief Networks (DBNs)', 'Autoencoders']
  Applications: ['Image recognition', 'Speech recognition', 'Natural language processing', 'Medical image analysis', 'Recommender systems', 'Autonomous vehicles', 'Game playing', 'Financial fraud detection', 'Drug d

Notice that `enum` has `valid_technologies`, so as we expand the urls, the number of topics would increase automatically.

In [ ]:
#bound technologies to what is in structured_outputs
valid_technologies = [item for item in structured_outputs.keys()]

#Define function schema for OpenAI
compare_technologies_schema = {
    "type": "function",
    "function": {
        "name": "compare_technologies",
        "description": "Compare two technologies",
        "parameters": {
            "type": "object",
            "properties": {
                "tech1": {
                    "type": "string",
                    "description": "The first technology to compare",
                    "enum": valid_technologies
                },
                "tech2": {
                    "type": "string",
                    "description": "The second technology to compare",
                    "enum": valid_technologies
                }
            },
            "required": ["tech1", "tech2"]
        }
    }
}
trace_evolution_schema = {
    "type": "function",
    "function": {
        "name": "trace_evolution",
        "description": "Trace the historical evolution of a given technology",
        "parameters": {
            "type": "object",
            "properties": {
                "technology": {
                    "type": "string",
                    "description": "The technology to trace",
                    "enum": valid_technologies
                }
            },
            "required": ["technology"]
        }
    }
}

In [ ]:
valid_technologies

['Agentic AI',
 'Reinforcement Learning',
 'Machine Learning',
 'Deep Learning',
 'Natural Language Processing']

# Part 4: Integration and Testing

In [ ]:
def demonstrate_function_calling(natural_queries:List[str]) -> List[str]:
    """
    Demonstrate function calling with natural language weather queries.

    This shows the complete flow:
    Natural Language → Function Call → API Response → Structured Data → Natural Answer
    """
    if not client:
      print("❌ OpenAI client not initialized. Exiting.")
      return

    results = []
    for query in natural_queries:
        print(f"🔍 Query: {query}")
        try:
          # Step 1: Send query to OpenAI with function definition
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",

                        "content": f"""You are a helpful research assistant. You have access to detailed data for these technologies: {valid_technologies}.
                        Use the relevant function to answer the question.
                        When users ask about relationships between technologies, use the compare_technologies function.
                        When users ask about the history or evolution of a technology, use the trace_evolution function.
                        The functions can handle technologies not in the database by providing general analysis.
                        Always try to use the appropriate function for comparison or evolution questions."""
                    },
                    {
                        "role": "user",
                        "content": query
                    }, # Added comma here
                ],
                tools=[compare_technologies_schema, trace_evolution_schema], #Available functions
                tool_choice="auto",  # Let the model decide when to call functions
            )
            response_message = response.choices[0].message
            # Step 2: Check if the model wants to call a function
            if response_message.tool_calls:
              tool_call = response_message.tool_calls[0]
              function_name = tool_call.function.name
              function_args = json.loads(tool_call.function.arguments)

              print(f"🔧 LLM decided to call: {function_name}({function_args})")
            # Step 3: Execute the function
              if function_name == "compare_technologies":
                result = compare_technologies(**function_args)
              elif function_name == "trace_evolution":
                result = trace_evolution(**function_args)
              if result:
                try:
                  response = client.chat.completions.create(
                      model="gpt-4o-mini",
                      messages=[
                          {
                              "role": "system",
                              "content": f"You are a helpful answering assistant. Use the following context {result} to answer the user query. If you dont know the answer, just say you dont know."
                          },
                          {
                              "role": "user",
                              "content": query
                          }, # Added another comma here
                      ],
                  )
                  answer = response.choices[0].message.content
                  print(answer)
                  results.append(answer)
                except Exception as e:
                  print(f"⚠️ Error generating final answer: {e}")
                  results.append(f"⚠️ Error generating final answer: {e}")
                  continue
            else:
              print("⚠️ No function supports query")
              results.append("No function supports query")
              continue
        except Exception as e:
          print(f"⚠️ Error processing query: {e}")
          results.append(f"⚠️ Error generating final answer: {e}")
          continue
    return results

In [ ]:
results = demonstrate_function_calling(
    [
        "What is the evolution for Reinforcement Learning?",
        "How is machine learning and deep learning related?",
        "Are agentic AI related to natural language processing?",
        "How should I connect reinforcement learning with deep learning?"
    ])

🔍 Query: What is the evolution for Reinforcement Learning?
🔧 LLM decided to call: trace_evolution({'technology': 'Reinforcement Learning'})
The evolution of Reinforcement Learning (RL) can be traced through several key developments over the years:

- **1950s**: Initial concepts of reinforcement learning were developed, primarily in the fields of robotics and control theory.
- **1980**: The introduction of Temporal Difference (TD) learning marked a significant advancement in RL, allowing for learning from temporally extended sequences of actions.
- **1992**: The Q-learning algorithm was proposed, providing a framework for learning the value of actions in a given state without needing a model of the environment.
- **2015**: The Deep Q-Network (DQN) was introduced, showcasing successful applications of reinforcement learning techniques in playing Atari games, utilizing deep learning methods to enhance performance.

Throughout this evolution, several major contributors have played signific

## Error handling for non related questions
For questions outside of the scope, No function will support query, it will still perform query for correct questions.

In [ ]:
results[1]

'Machine learning and deep learning are closely related, with deep learning being a subfield of machine learning. \n\nMachine learning focuses on creating systems that learn from data and improve their performance over time, using various techniques such as supervised learning, unsupervised learning, reinforcement learning, decision trees, and support vector machines.\n\nDeep learning, on the other hand, specifically utilizes multilayered neural networks to perform tasks such as classification and regression. It mimics the way human brains operate by stacking artificial neurons to process large amounts of data, leading to significant advancements in tasks like image recognition, speech recognition, and natural language processing.\n\nIn summary, all deep learning techniques are part of the broader machine learning field, but not all machine learning techniques are deep learning.'

In [ ]:
bad_results = demonstrate_function_calling(
    [
        "What did I have for dinner?",
        "Is AI good",
        "How is machine learning and deep learning related?",
    ])

🔍 Query: What did I have for dinner?
⚠️ No function supports query
🔍 Query: Is AI good
⚠️ No function supports query
🔍 Query: How is machine learning and deep learning related?
🔧 LLM decided to call: compare_technologies({'tech1': 'Machine Learning', 'tech2': 'Deep Learning'})
Machine learning and deep learning are closely related concepts in the field of artificial intelligence, but they differ in their scope and techniques. 

- **Machine Learning** is a broader field that encompasses a variety of techniques and approaches for creating systems that can learn from data and make predictions. It includes methods like supervised learning, unsupervised learning, reinforcement learning, decision trees, support vector machines, neural networks, and clustering.

- **Deep Learning**, on the other hand, is a specialized subset of machine learning. It specifically focuses on using multilayered neural networks (also known as deep neural networks) to perform complex tasks such as classification an

In [ ]:
bad_results[2]

'Machine learning and deep learning are closely related concepts in the field of artificial intelligence, but they differ in their scope and techniques. \n\n- **Machine Learning** is a broader field that encompasses a variety of techniques and approaches for creating systems that can learn from data and make predictions. It includes methods like supervised learning, unsupervised learning, reinforcement learning, decision trees, support vector machines, neural networks, and clustering.\n\n- **Deep Learning**, on the other hand, is a specialized subset of machine learning. It specifically focuses on using multilayered neural networks (also known as deep neural networks) to perform complex tasks such as classification and regression. Deep learning effectively handles high-dimensional data and has led to significant advancements in areas such as computer vision, speech recognition, and natural language processing.\n\nIn summary, while all deep learning is machine learning, not all machine 

In [ ]:
bad_results[1]

'No function supports query'

# Results and Analysis



## Display of scrapped data
Below is the scrapped data, we jsut show one topic, Agentic AI

In [ ]:
for key, value in scrapper_result[0].items():
  if key != "raw_content":
    print(f"{key} : {value}")
  else:
    print(f"{key} : {value[:2000] + "..."}")

url : https://en.wikipedia.org/wiki/Agentic_AI
title : Agentic AI - Wikipedia
raw_content : # Agentic AI
4 languages
  * [Català](https://ca.wikipedia.org/wiki/IA_ag%C3%A8ntica "IA agèntica – Catalan")
  * [فارسی](https://fa.wikipedia.org/wiki/%D9%87%D9%88%D8%B4_%D9%85%D8%B5%D9%86%D9%88%D8%B9%DB%8C_%D8%AE%D9%88%D8%AF%D9%85%D8%AE%D8%AA%D8%A7%D8%B1 "هوش مصنوعی خودمختار – Persian")
  * [한국어](https://ko.wikipedia.org/wiki/%EC%97%90%EC%9D%B4%EC%A0%84%ED%8B%B1_AI "에이전틱 AI – Korean")
  * [Русский](https://ru.wikipedia.org/wiki/%D0%90%D0%B3%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D0%98%D0%98 "Агентный ИИ – Russian")

  * [Article](https://en.wikipedia.org/wiki/Agentic_AI "View the content page \[alt-c\]")
  * [Talk](https://en.wikipedia.org/wiki/Talk:Agentic_AI "Discuss improvements to the content page \[alt-t\]")

English
  * [Read](https://en.wikipedia.org/wiki/Agentic_AI)
  * [Edit](https://en.wikipedia.org/w/index.php?title=Agentic_AI&action=edit "Edit this page \[alt-e\]")
  * [View history]

## Structured extraction results

In [ ]:
for key, value in structured_outputs.items():
  print(f"AI subtopic: {key}")
  print(f"Summary: {value.summary}")
  print(f"Evolution Timeline: {value.evolution_timeline}")
  print(f"Key Innovations: {value.key_innovations}")
  print(f"Major Contributors: {value.major_contributors}")
  print(f"Main Techniques: {value.main_techniques}")
  print(f"Applications: {value.applications}")
  print("\n")

AI subtopic: Agentic AI
Summary: Agentic AI is a class of artificial intelligence focused on autonomous systems that can make decisions and perform tasks with or without human intervention, utilizing various AI techniques such as natural language processing and machine learning.
Evolution Timeline: 1998: The term 'agent-based process management system' is first used to describe autonomous agents for business process management.
Key Innovations: Introductions of intelligent agents, autonomous systems, and advancements in algorithmic and human-like creative processes.
Major Contributors: MIT, TechCrunch, companies involved in automation and software development.
Main Techniques: ['Natural Language Processing', 'Machine Learning', 'Computer Vision', 'Robotic Process Automation']
Applications: ['Software Development', 'Customer Support', 'Cybersecurity', 'Business Intelligence', 'Web Browsing']


AI subtopic: Reinforcement Learning
Summary: Reinforcement Learning (RL) is a subfield of mach

## Sample function calling converstaions

In [ ]:
results = demonstrate_function_calling(
    [
        "What is the evolution for Reinforcement Learning?",
        "How is machine learning and deep learning related?",
        "Are agentic AI related to natural language processing?",
        "How should I connect reinforcement learning with deep learning?"
    ])

🔍 Query: What is the evolution for Reinforcement Learning?
🔧 LLM decided to call: trace_evolution({'technology': 'Reinforcement Learning'})
The evolution of Reinforcement Learning (RL) can be traced through several key developments over the decades:

- **1950s**: Initial concepts of reinforcement learning are developed, particularly in the context of robotics and control theory. This period laid the groundwork for understanding how agents can learn to make decisions based on their interactions with environments.

- **1980**: The concept of Temporal Difference Learning is introduced, which allows agents to learn optimal policies by estimating the value of states based on predictions of future rewards.

- **1992**: The Q-learning algorithm is proposed, offering a model-free approach for agents to learn the value of actions using a value function.

- **2015**: The Deep Q-Network (DQN) is developed, demonstrating successful applications of reinforcement learning in playing complex video ga

In [ ]:
for reply in results:
  print(reply)

The evolution of Reinforcement Learning (RL) can be traced through several key developments over the decades:

- **1950s**: Initial concepts of reinforcement learning are developed, particularly in the context of robotics and control theory. This period laid the groundwork for understanding how agents can learn to make decisions based on their interactions with environments.

- **1980**: The concept of Temporal Difference Learning is introduced, which allows agents to learn optimal policies by estimating the value of states based on predictions of future rewards.

- **1992**: The Q-learning algorithm is proposed, offering a model-free approach for agents to learn the value of actions using a value function.

- **2015**: The Deep Q-Network (DQN) is developed, demonstrating successful applications of reinforcement learning in playing complex video games, particularly Atari games. This marked a significant leap in the use of deep learning techniques in RL.

Throughout this evolution, seve

# Research Insights and Reflection
## Key Insights:

###Interconnected Nature of AI Tehcnologies:
Although AI subtopics are often introduced individually, these methods are constantly being used in conjuction to create more powerful products. For example, q learning combined with deep neural netowrk results in Deep Q-Networks(DQN), Agentic AI incorperates techniques like natural language processing. This interconnectedness shows that modern AI solutions are rarely built on a single technique but rather represent convergent evolution of multiple AI approaches.

###Educational Value of Structured Analysis:
From a student's perspective, this system can easily create connections without requiring students to read through entire Wikipedia articles.The structured format provides quick access to evolution timelines, key innovations, and major contributors, giving students both breadth and depth in understanding how different AI fields developed and relate to each other.

###Historical Context Matters:
The evolution analysis revealed that understanding the timeline of AI development helps appreciate current capabilities. For instance, seeing how Reinforcement Learning evolved from 1950s psychology principles to 2020s sophisticated algorithms provides crucial context for understanding why certain approaches work and others don't.

## Design decisions and code structure explanations
###Data Storage Strategy
The biggest architectural decision was how to store and query structured outputs. Initially, I considered implementing a query analysis system to extract main topics from a given list, which would have required additional API calls for topic identification. However, I ultimately chose to store structured outputs in a dictionary with the main topic as the key, using enum constraints to bound function calls to available topics. This design eliminated one API call to OpenAI while maintaining clean, predictable data access patterns. <br>

###Prompt Engineering Strategies
I had to do some prompt engineering to make it more likely for methods to be used as I realized a generic "Choose the relevant function" results in no functions being called for some of the questions. <br>

###Error Handling
The system includes robust error handling that continues processing queries even when individual queries fail, and provides fallback responses when functions aren't called or data isn't available. It also prints which part of the code has errors.

## Challenges faced and solutions implemented
Throughout the assignment, I struggled with deciding the general flow of a relatively complex task. For example, should chunking be used, how to store the outputs, what format to use.

Solution: I adopted an iterative and wishful thinking approach during the desings. Suppose I already have two correct and perfect functions to analyze a given technology, how would I showcase the demo. This really helped with the details and I had to keep rethinking and rewriting certain parts of the code to fit. Intially structured_outputs was just a list, but if I need to query, a dictionary would make much more sense. Then test cases where I expect certain output and how do enhance prompt engineering to make sure the right functions are called.

## Meaningful Patterns and Connections Uncovered
###Overlap of technologies
The analysis revales how different AI fields often overlap and require similar techniques and overlap in scope. Technologies like Agentic AI demonstrate how modern AI applications integrate techniques from multiple traditional AI subfields, suggesting that future AI development will be increasingly interdisciplinary. This assitance helps to connect the dots easily, and is a fruit for thought as I learn more about my masters

##What you learned from this assignment
###Technical Skills:
- Scraping with crawl4ai
- Structured outputs with LLM
- function calling, function schemas and prompt engineering
- error handling and strategies for robust systems
- designing an AI system
###Domain Knowledge:
- Gained deeper appreciation for the interconnected nature of AI technologies
- Understanding how historical context enriches current AI discussions
- Recognition that effective AI education need to highlight relationships between concepts, not just individual topic explanations



#AI Tool Declaration
I used GPT-5 and Claude Sonnet 4 to generate ideas, seek advice for unsure code, debug and consolidate my ideas. I am responsible for the content and quality of the submitted work.